# Project 3. Limited time training

 In project 3, you're going to improve your network under the limitation of time, specifically 10 minutes. You can borrow any structure that you consider viable, but pretrained models are **NOT allowed**.

**Hint**: Most of the popular networks can't converge in 10 minutes, so if you decide to borrow any, you'd have to trim the network a little to fit into the time frame.

## 1. Preparation

This project is time limited, we need to make sure that your hardwares are exactly the same, try the following code, if you see 'GPU correct, proceed.', you can continue the project, otherwise, click the dropdown at the upper right corner besides the status bar (RAM, Disk), and go to 'Manage sessions', and terminate the current session, then reconnect, and try the code again, repeat until you see the right message.

In [ ]:
!if [ $(nvidia-smi | grep Tesla | tr -s " " | cut -d' ' -f4) == 'T4' ]; then echo "GPU correct, proceed."; else echo "GPU incorrect, please terminate this session and reconnect again."; fi

## 2. Before training

For dataset, we continue to use CIFAR10, in this part, you can configure all of your settings, including data augmentation, model structure, optimizer etc., but don't put any major calculation part in.

### 2.1 Dataset

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
from torch.utils.data import DataLoader
import numpy as np
import torchvision.transforms as tt

stats = ((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
train_tfms = tt.Compose([tt.RandomCrop(32, padding=4, padding_mode='reflect'), 
                         tt.RandomHorizontalFlip(), 
                         tt.ToTensor(), 
                         tt.Normalize(*stats,inplace=True)])
valid_tfms = tt.Compose([tt.ToTensor(), tt.Normalize(*stats)])

# cifar10_train = torchvision.datasets.CIFAR10("CIFAR10", train=True, transform=lambda x: np.array(x, dtype=np.float32).transpose(2,0,1),
#                                          target_transform=None, download=True)
# cifar10_test = torchvision.datasets.CIFAR10("CIFAR10", train=False, transform=lambda x: np.array(x, dtype=np.float32).transpose(2,0,1),
#                                          target_transform=None, download=True)

cifar10_train = torchvision.datasets.CIFAR10("CIFAR10", train=True, transform=train_tfms,
                                         target_transform=None, download=True)
cifar10_test = torchvision.datasets.CIFAR10("CIFAR10", train=False, transform=valid_tfms,
                                         target_transform=None, download=True)

Files already downloaded and verified
Files already downloaded and verified


### 2.2 Model

**Hint**: Even though pretrained models are not allowed in your final version, you can use pretrained models to determine how well the structure CAN perform and use this as part of your analysis as to determining network structure.

**Model definition**

In [3]:
# resnet-9 5GB - 88.5
# resnet-18 10GB - not finished
# my net 3.5GB 86

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.nn.init as init


def conv_block(in_channels, out_channels, pool=False, eps=0.00001, mon=0.1, dropout=0):
    layers = [nn.Conv2d(in_channels, out_channels, kernel_size=3,stride=1, padding=1, bias=False,padding_mode='zeros'),
              nn.BatchNorm2d(out_channels,eps,mon),
              nn.ReLU(inplace=True)]
    if pool: layers.append(nn.MaxPool2d(2))
    # if dropout > 0: layers.append(nn.Dropout(dropout))
    return nn.Sequential(*layers)

class ResNet(nn.Module):
    def __init__(self, in_channels, num_classes):
        super(ResNet, self).__init__()

        self.conv1 = conv_block(in_channels, 64, eps=1e-5, mon=0.9)
        self.conv2 = conv_block(64, 128, eps=1e-5, mon=0.9)
        self.res1 = nn.Sequential(conv_block(128, 128, eps=1e-5, mon=0.9), conv_block(128, 128, eps=1e-5, mon=0.9))

        self.conv3 = conv_block(128, 256, pool=True, eps=1e-5, mon=0.9)
        self.conv4 = conv_block(256, 512, pool=True, eps=1e-5, mon=0.9)
        self.res2 = nn.Sequential(conv_block(512, 512, eps=1e-5, mon=0.9), conv_block(512, 512, eps=1e-5, mon=0.9))

        self.classifier = nn.Sequential(nn.AdaptiveMaxPool2d((1,1)),
                                        nn.Flatten(),
                                        # nn.Dropout(0.1),
                                        nn.Linear(512, num_classes, bias=False))

    def forward(self, xb):
        out = self.conv1(xb)
        out = self.conv2(out)
        out = self.res1(out) + out
        out = self.conv3(out)
        out = self.conv4(out)
        out = self.res2(out) + out
        out = self.classifier(out)
        out *= 0.125
        return out

net = ResNet(3, 10).cuda()



RuntimeError: Unexpected error from cudaGetDeviceCount(). Did you run some cuda functions before calling NumCudaDevices() that might have already set an error? Error 803: system has unsupported display driver / cuda driver combination

**Analysis tools**

In [ ]:
# @torch.compile  
def testset_precision(net, testset):
    net.eval()
    dl = DataLoader(testset, batch_size=512)
    total_count = 0
    total_correct = 0
    for data in dl:
        inputs = data[0].cuda()
        targets = data[1].cuda()
        outputs = net(inputs)
        predicted_labels = outputs.argmax(dim=1)
        comparison = predicted_labels == targets
        total_count += predicted_labels.size(0)
        total_correct += comparison.sum()
    net.train()
        
    return int(total_correct) / int(total_count)

print(f'Inital precision: {testset_precision(net, cifar10_test)}')

from IPython import display
import matplotlib.pyplot as plt
import time

class DrawingBoard:
    def __init__(self, names, time_slot=60):
        self.start_time = time.time()
        self.time_slot = time_slot
        self.annotations = []
        self.data = {}
        for name in names:
            self.data[name] = []
    
    def update(self, data_dict):
        for key in data_dict:
            self.data[key].append(data_dict[key])
        current_time = time.time() - self.start_time
        idx = len(self.data[key]) -1
        if len(self.annotations) == 0:
            if current_time > self.time_slot:
                self.annotations.append((idx, current_time))
        elif current_time - (self.annotations[-1][1]//self.time_slot)*self.time_slot > self.time_slot:
            self.annotations.append((idx, current_time))
    
    def draw(self):
        all_keys = list(self.data.keys())
        fig, ax = plt.subplots(nrows=1, ncols=len(all_keys))
        fig.set_figwidth(20)
        for idx in range(len(all_keys)):
            ax[idx].plot(self.data[all_keys[idx]])
            ax[idx].grid()
            ax[idx].set_title(all_keys[idx])
            for an in self.annotations:
                ax[idx].annotate(f'{int(an[1])}s', xy=(an[0], self.data[all_keys[idx]][an[0]]),
                    xytext=(0, -40), textcoords="offset points",
                    va="center", ha="left",
                    bbox=dict(boxstyle="round", fc="w"),
                    arrowprops=dict(arrowstyle="->"))
        display.clear_output(wait=True)
        plt.show()

In [ ]:
# modify as you need
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=0.001)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=3, gamma=0.5) # 3, 0.85
dataloader = DataLoader(cifar10_train, batch_size=512, shuffle=True, num_workers=2, pin_memory=True)

## 3. Start Training

This is the training part, take advantage of everything you know. In addition to the tips last time, you can also consider data augmentation, short cut structure in ResNet etc.

Basically at this point, nothing is limited except time and pretrained models.

In [ ]:
# modify as you need
import time
import signal
class TimeLimitation:
    def __init__(self, limit):
        self.limit = limit

    def __enter__(self):
        def handler(signum, frame):
            raise NotImplementedError('Time\'s up')
        signal.signal(signal.SIGALRM, handler)
        signal.alarm(self.limit)

    def __exit__(self, exc_type, exc_val, exc_tb):
        signal.alarm(0)


with TimeLimitation(60*60*2): # don't forget the indentation
    print('start training')
    db = DrawingBoard(['training_loss', 'testset_precision'], time_slot=60)
    for epoch in range(500):
        for iter, data in enumerate(dataloader):
            inputs = data[0].cuda()
            targets = data[1].cuda()
            outputs = net(inputs)

            optimizer.zero_grad()
            loss = criterion(outputs, targets)
            loss.backward()
            nn.utils.clip_grad_value_(net.parameters(), 0.1)
            optimizer.step()

            if iter % 20 == 0:
                num = testset_precision(net, cifar10_test)
                print(num)
                db.update({'training_loss':loss.item(), 'testset_precision': num})
                db.draw()
        scheduler.step()

    print('finished')

# Marking Scheme

*   Code implementation: 30%


> * Successfully run through the whole project: 10%
> * Code implementation quality: 20% (including performance)

*   PDF report: 50%

> * Overview, introduction to the network structure etc.: 10%
> * In-depth analysis and improvements: 40%

*   Presentation: 20%

> * Overall demonstration: 10%
> * Unfolding analysis: 10%